In [91]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression as LR
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor as KNN

In [92]:
train = pd.read_csv("train.csv")
target = train['averageRating']
#indices = np.random.choice(train.index, size=len(train) // 10, replace=False)

# Разделяем DataFrame на два на основе выбранных индексов
#test_kol = train.loc[indices]
#target_kol = test_kol['averageRating']
#train = train.drop(indices)
target = train['averageRating']

In [93]:
train.describe()

,Id,isAdult,startYear,endYear,runtimeMinutes,numVotes,mainDirectorBirthYear,mainDirectorDeathYear,mainWriterBirthYear,mainWriterDeathYear,titleLength,genderVoters,worldPromotion,Downloads,Cash,averageRating
count,2000.000000,2000.000000,2000.000000,45.000000,1594.000000,2000.000000,1023.000000,403.000000,1006.000000,391.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,999.500000,0.011000,1996.948500,1997.622222,58.572773,816.366000,1943.176931,1993.468983,1942.867793,1985.202046,17.454000,435.068000,0.493500,4148.150000,30987.277000,6.883400
std,577.494589,0.104329,21.131614,18.434514,41.789627,10587.436793,26.718782,21.173200,34.214969,37.361472,9.816622,6531.143909,0.500083,3198.266535,18211.802598,1.335023
min,0.000000,0.000000,1910.000000,1954.000000,1.000000,5.000000,1868.000000,1925.000000,1564.000000,1616.000000,1.000000,0.000000,0.000000,-1.000000,43.000000,1.000000
25%,499.750000,0.000000,1987.000000,1986.000000,24.000000,10.000000,1924.000000,1981.000000,1924.000000,1974.000000,11.000000,4.000000,0.000000,1065.250000,15174.750000,6.200000
50%,999.500000,0.000000,2005.000000,2005.000000,50.000000,27.000000,1947.000000,2000.000000,1947.500000,1994.000000,15.000000,11.000000,0.000000,3903.000000,30338.000000,7.100000
75%,1499.250000,0.000000,2012.000000,2014.000000,90.000000,103.000000,1964.000000,2010.000000,1967.000000,2007.000000,21.000000,45.000000,1.000000,7023.250000,46775.250000,7.800000
max,1999.000000,1.000000,2018.000000,2017.000000,418.000000,418689.000000,2000.000000,2018.000000,2000.000000,2018.000000,89.000000,264111.000000,1.000000,10000.000000,62776.000000,10.000000


In [94]:
def categories(df):
  mapping = {}
  counter = 10

  for category in df:
    if category not in mapping:
        mapping[category] = counter
        counter += 10
  return df.map(mapping)

In [95]:
def rem(X):
  X = X.str.replace(" ", '')
  X = X.str.replace("'", '')
  X = X.str.removeprefix("[")
  X = X.str.removesuffix("]")
  X = X.replace(" ", '')
  X = X.str.split(',', expand = True)
  return pd.DataFrame(X)

def all_genres(X):
  X = rem(X)
  genres = pd.unique(X.values.ravel())
  genres = genres[genres!='nan']
  genres = genres[genres!='None']
  return genres

all_genres = all_genres(train['genres'])

all_genres

array(['Drama', 'Horror', 'Short', 'Action', 'Animation', 'Crime',
       'Thriller', 'Musical', 'Mystery', 'Biography', 'Documentary',
       'Comedy', 'History', 'Family', 'Romance', 'Western', 'Adventure',
       'Fantasy', 'Sci-Fi', 'Sport', 'Music', 'Adult', 'Reality-TV',
       'War', 'News', 'Talk-Show', 'Game-Show', 'Film-Noir'], dtype=object)

In [96]:
def genres_to_bin(X):
  res = pd.DataFrame()
  for i in all_genres:
    res[i] = X.eq(i).any(axis=1).astype(int)
  return res.fillna(0)



In [97]:
titles = np.unique(train['titleType'].values)
def titles_to_bin(X):
  res = pd.DataFrame()
  for i in titles:
    res[i] = X.eq(i).astype(int)

  return res

In [98]:
def all_professions(X):
  X = X.str.split(',', expand = True)

  res = np.array(pd.unique(X.values.ravel())).astype(str)
  res = res[res!='nan']
  res = res[res!='None']
  return res.astype(str)

def professions(X, all_prof, ch):
  X = X.str.split(',', expand = True)
  res = pd.DataFrame()
  for i in all_prof:
    res[f'{i}:{ch}'] = X.eq(i).any(axis=1).astype(int)

  return res.fillna(0)

l = all_professions(train['MainWriterPrimaryProfession'])
professions(train['MainWriterPrimaryProfession'], l, 'w')

,producer:w,director:w,writer:w,art_department:w,actor:w,assistant_director:w,miscellaneous:w,editor:w,soundtrack:w,visual_effects:w,...,executive:w,editorial_department:w,production_designer:w,stunts:w,make_up_department:w,location_management:w,costume_designer:w,set_decorator:w,costume_department:w,casting_department:w
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
def standardize_column(column):
    mean = column.mean()
    std = column.std()
    if std == 0:
      return column
    else:
      return (column - mean) / std


class Preprocessing:
    def fit(self, X, y):
      self.knn_len = KNN(15).fit(X[['titleLength']].interpolate(), y)
      self.knn_st_year = KNN(30).fit(X[['startYear']].interpolate(), y)
      self.knn_time = KNN(30).fit(X[['runtimeMinutes']].interpolate(), y)
      X['age_sub'] = X['mainDirectorBirthYear'].sub(X['mainWriterBirthYear'])
      X['age_sub2'] = X['mainDirectorBirthYear'].sub(X['startYear'])
      self.knn_age = KNN(15).fit(X[['age_sub']].interpolate().fillna(0), y)
      self.knn_age2 = KNN(15).fit(X[['age_sub2']].interpolate().fillna(0), y)
      self.knn_promotion = KNN(100).fit(X[['worldPromotion']].interpolate(), y)
      self.prof_d = all_professions(X['MainWriterPrimaryProfession'])
      self.prof_w = all_professions(X['MainDirectorPrimaryProfession'])



    def all_paramets(self, X):
        res = pd.DataFrame()
        res['startYear']= X['startYear']
        res['runtimeMinutes']= X['runtimeMinutes']
        res['runtimeMinutes**3']= X['runtimeMinutes'].mul(X['runtimeMinutes']).mul(X['runtimeMinutes'])
        res['runtimeMinutes*startYear']= X['startYear'].mul(X['runtimeMinutes'])
        res['numVotes']= X['numVotes']
        res['numVotes*startYear']= res['numVotes'].mul(res['startYear'])
        res['runtimeMinutes*startYear']= res['runtimeMinutes'].mul(res['startYear'])
        res['runtimeMinutes*numVotes']= res['runtimeMinutes'].mul(res['numVotes'])
        res['numVotes>']= (X['numVotes']>100000).astype(int)
        res['genderVoters']= X['genderVoters']
        res['Downloads']= X['Downloads']
        res['Cash']= X['Cash']
        res['Cash**2']= X['Cash'].mul(res['Cash'])
        res['Cash*Downloads'] =  res['Cash'].mul(res['Downloads'])
        res['voges_mul'] = res['numVotes'].sub(res['genderVoters'])
        res['mainWriterBirthYear'] = X['mainWriterBirthYear'].fillna(1945)
        res['mainDirectorBirthYear'] = X['mainDirectorBirthYear'].fillna(1945)
        res['age_sub'] = res['mainDirectorBirthYear'].sub(res['mainWriterBirthYear'])
        res['age_sub2'] = res['mainDirectorBirthYear'].sub(res['startYear'])
        res['titleLength']= X['titleLength']
        res['titleLength**2']= X['titleLength'].mul(X['titleLength'])

        res['title'] = categories(X['titleType'])
        res['prefVotersCountry'] = categories(X['prefVotersCountry'])
        res['firstShownCountry'] = categories(X['firstShownCountry'])
        res['firstShownCountry*prefVotersCountry'] = res['firstShownCountry'].mul(res['prefVotersCountry'])


        res['knn_title_len'] = self.knn_len.predict(X[['titleLength']].interpolate())
        res['knn_st_year'] = self.knn_st_year.predict(X[['startYear']].interpolate())
        res['knn_time'] = self.knn_time.predict(X[['runtimeMinutes']].interpolate())
        res['knn_worldPromotion'] = self.knn_promotion.predict(X[['worldPromotion']].interpolate())
        res['knn_age'] =self.knn_age.predict(res[['age_sub']].interpolate().fillna(0))
        res['knn_age2'] =self.knn_age2.predict(res[['age_sub2']].interpolate().fillna(0))






        res['cash_log'] = res['Cash'].apply(lambda x: np.log(x))
        res['Downloads_log'] = res['Downloads'].apply(lambda x: np.log(x))


        res = pd.concat([res, titles_to_bin(X['titleType'])], axis=1)



        res['isAdult']= X['isAdult']
        res['worldPromotion']= X['worldPromotion']
        res['change_title'] = X['primaryTitle'] == X['originalTitle']
        res = pd.concat([res, genres_to_bin(rem(X["genres"]))], axis=1)
        #res = pd.concat([res, professions(X['MainWriterPrimaryProfession'], self.prof_w, "w")], axis=1)
        #res = pd.concat([res, professions(X['MainDirectorPrimaryProfession'], self.prof_w,'d')], axis=1)

        res = res.interpolate()




        normalized_res = res.apply(standardize_column)

        return res






In [157]:
p = Preprocessing()
p.fit(train, target)
train_ = p.all_paramets(train)
train_

,startYear,runtimeMinutes,runtimeMinutes**3,runtimeMinutes*startYear,numVotes,numVotes*startYear,runtimeMinutes*numVotes,numVotes>,genderVoters,Downloads,...,Sci-Fi,Sport,Music,Adult,Reality-TV,War,News,Talk-Show,Game-Show,Film-Noir
0,2009,23.000000,1.216700e+04,46207.000000,32,64288,7.360000e+02,0,17.0,7658,...,0,0,0,0,0,0,0,0,0,0
1,2009,112.000000,1.404928e+06,225008.000000,13110,26337990,1.468320e+06,0,6513.0,2703,...,0,0,0,0,0,0,0,0,0,0
2,2010,7.000000,3.430000e+02,14070.000000,5,10050,3.500000e+01,0,2.0,7541,...,0,0,0,0,0,0,0,0,0,0
3,2000,38.333333,3.436623e+05,75703.333333,6,12000,1.538333e+03,0,1.0,4446,...,0,0,0,0,0,0,0,0,0,0
4,1986,69.666667,6.869817e+05,137336.666667,10,19860,3.041667e+03,0,7.0,4671,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1991,40.000000,1.120000e+05,79060.000000,19,37829,2.500000e+02,0,9.0,6755,...,0,0,0,0,0,0,0,0,0,0
1996,2005,20.000000,8.000000e+03,40100.000000,10,20050,2.000000e+02,0,9.0,6993,...,0,0,0,0,0,0,0,0,0,0
1997,2013,44.000000,8.518400e+04,88572.000000,10,20130,4.400000e+02,0,1.0,-1,...,0,0,0,0,0,0,0,0,0,0
1998,2017,48.000000,1.105920e+05,96816.000000,103,207751,4.944000e+03,0,53.0,-1,...,0,0,0,0,0,0,0,0,0,0


In [101]:
train_.columns.size

72

In [158]:
col = np.array(train_.columns)

def new_col(col, s):
  col = col[col!=s]
  return col

In [159]:
cur_paraments = pd.DataFrame()

def new_df(cur_paraments, s):
  cur_paraments = pd.concat([cur_paraments ,pd.DataFrame(train_[s])], axis = 1)
  return cur_paraments

In [160]:
from sklearn.metrics import mean_squared_error as mse

def search(col, cur_paraments):
  l = {}
  for column in col:
      train_d = pd.concat([cur_paraments ,pd.DataFrame(train_[column])], axis = 1)
      rmses = []
      for i in range(15):
        X_train, X_test, y_train, y_test = train_test_split(train_d, target, test_size=0.2, random_state = i)
        model = LR().fit(X_train, y_train)
        preds = model.predict(X_test)
        rmses.append(mse(y_test, preds) ** 0.5)

      l[column]= np.mean(rmses)



  min_key = min(l, key=l.get)


  return min_key, l[min_key]


In [161]:
res = {}

for i in range(40):
  s, val = search(col, cur_paraments)
  col = new_col(col, s)
  cur_paraments = new_df(cur_paraments, s)

  #test_ = train_[cur_paraments.columns]
  #model = LR().fit(test_, target)

  #preds = model.predict(p.all_paramets(test_kol)[cur_paraments.columns])
  #res[i] = mse(target_kol, preds) ** 0.5
  res[i]=val






In [175]:
colums = cur_paraments.iloc[:, :25].columns
cur_paraments.columns.size

40

In [167]:
res

{0: 1.2222744579074463,
 1: 1.1937225538790555,
 2: 1.178113010291824,
 3: 1.1646630291487428,
 4: 1.15795786240309,
 5: 1.1512157228460433,
 6: 1.1477690385849648,
 7: 1.144248457301383,
 8: 1.1412807610402171,
 9: 1.1380405440028212,
 10: 1.135297898964944,
 11: 1.1327564969696438,
 12: 1.130675443227901,
 13: 1.1289118175395456,
 14: 1.1272837427406444,
 15: 1.1259834104847948,
 16: 1.1253576109144938,
 17: 1.1247871090893968,
 18: 1.124407112598284,
 19: 1.1240319036603916,
 20: 1.123849665883503,
 21: 1.1236178697860646,
 22: 1.1235917062614298,
 23: 1.1235779269008137,
 24: 1.1235888646773404,
 25: 1.1235942862845458,
 26: 1.1235625882956006,
 27: 1.1235838319920466,
 28: 1.1236092334801921,
 29: 1.1236867656991152,
 30: 1.1237823629890678,
 31: 1.1237823629919332,
 32: 1.1238995931935818,
 33: 1.1240830309254297,
 34: 1.1242409631969188,
 35: 1.1243949746036035,
 36: 1.1246186647440173,
 37: 1.1248689995034,
 38: 1.1251522528434965,
 39: 1.125418203949714}

In [176]:
p = Preprocessing()
p.fit(train, target)
train_ = p.all_paramets(train)[colums]
model = LR().fit(train_, target)

preds = model.predict(train_)
mse(target, preds) ** 0.5
preds

array([6.51240776, 5.94558107, 6.92383885, ..., 8.21610773, 7.94411954,
       6.55044188])

In [177]:
test = pd.read_csv("test.csv")

test_ = p.all_paramets(test)[colums]

preds = np.array(model.predict(test_))
np.savetxt('output.txt', preds, fmt='%s')


In [ ]:
test_

,knn_time,firstShownCountry,prefVotersCountry,Fantasy,worldPromotion,Horror,runtimeMinutes,Mystery,Talk-Show,Music,mainWriterBirthYear,numVotes>,titleLength**2,Animation
0,7.24,10,10,0,0,0,130.0,0,0,1,1885.0,0,16,0
1,6.68,20,20,0,0,0,88.5,0,0,0,1948.0,0,1156,0
2,6.54,30,30,0,1,0,47.0,1,0,0,1944.0,0,441,0
3,5.86,40,40,0,0,0,94.0,0,0,0,1945.0,0,784,0
4,6.32,50,50,0,1,0,101.0,0,0,0,1962.0,0,400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,6.66,310,1930,0,0,0,7.0,0,0,0,1897.0,0,324,1
1996,7.08,1040,1750,0,1,0,43.0,0,0,0,1945.0,0,841,1
1997,7.36,980,2150,0,0,0,28.0,0,0,0,1945.0,0,225,0
1998,6.50,1970,2170,0,0,0,100.0,0,0,1,1908.0,0,400,0
